In [241]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2025/Conferences.csv
/kaggle/input/march-machine-learning-mania-2025/SeedBenchmarkStage1.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/WSecondaryTourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2025/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-20

In [242]:
DATA_PATH = '/kaggle/input/march-machine-learning-mania-2025/'


In [243]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import xgboost as xgb
from scipy.interpolate import UnivariateSpline
from sklearn import preprocessing
from sklearn.metrics import brier_score_loss, log_loss
from sklearn.model_selection import KFold
from tqdm import tqdm

pd.set_option("display.max_column", 200)
# print(os.listdir("../input"))
xgb.__version__ # I used '1.2.0-SNAPSHOT'

'2.0.3'

In [244]:
import os
MTeamname = pd.read_csv(DATA_PATH + "MTeamSpellings.csv")
WTeamname = pd.read_csv(DATA_PATH + "WTeamSpellings.csv")
Melo = pd.read_csv("/kaggle/input/warrennolan-elo/Melo.csv")
Welo = pd.read_csv("/kaggle/input/welo-2025/WELO.csv")


In [245]:
print(Welo.head())


             Team Record      ELO  Rank  ELO Delta
0  South Carolina   30-3  1928.27     1        NaN
1            UCLA   30-2  1917.99     2        NaN
2           Texas   31-3  1909.62     3        NaN
3             USC   28-3  1893.50     4        NaN
4     Connecticut   31-3  1882.08     5        NaN


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [246]:
import re

def normalize_team_name(name):
    return re.sub(r'[^a-zA-Z0-9]', '', str(name)).lower()

In [247]:

MTeamname["Normalized"] = MTeamname["TeamNameSpelling"].apply(normalize_team_name)
WTeamname["Normalized"] = WTeamname["TeamNameSpelling"].apply(normalize_team_name)

Melo["Normalized"] = Melo["Team"].apply(normalize_team_name)
Welo["Normalized"] = Welo["Team"].apply(normalize_team_name)

In [248]:
# 'ELO Delta' 컬럼 삭제
if 'ELO Delta' in Melo.columns:
    Melo.drop(columns=['ELO Delta'], inplace=True)
if 'ELO Delta' in Welo.columns:
    Welo.drop(columns=['ELO Delta'], inplace=True)


In [249]:
MTeamname.rename(columns=lambda x: x.strip(), inplace=True)
Melo.rename(columns=lambda x: x.strip(), inplace=True)
Welo.rename(columns=lambda x: x.strip(), inplace=True)

# 데이터 확인 (오류가 발생하는지 테스트)
print(MTeamname.tail(5))
print(Welo.tail(5))

      TeamNameSpelling  TeamID       Normalized
1172     youngstown st    1464     youngstownst
1173    youngstown st.    1464     youngstownst
1174  youngstown state    1464  youngstownstate
1175     youngstown-st    1464     youngstownst
1176  youngstown-state    1464  youngstownstate
                     Team Record     ELO  Rank          Normalized
357         UNC Asheville   2-27  903.71   358        uncasheville
358        Delaware State   1-24  888.91   359       delawarestate
359   Arkansas-Pine Bluff   1-25  887.88   360   arkansaspinebluff
360           Long Island   6-24  884.35   361          longisland
361  South Carolina State   2-29  856.91   362  southcarolinastate


In [250]:
Melo = pd.merge(
    Melo, 
    MTeamname[['TeamID', 'Normalized']], 
    on="Normalized", 
    how="left"
)

Welo = pd.merge(
    Welo, 
    WTeamname[['TeamID', 'Normalized']], 
    on="Normalized", 
    how="left"
)
Melo["TeamID"] = Melo["TeamID"].fillna(0).astype(int)  # 또는 astype('Int64') 사용 가능
Welo["TeamID"] = Welo["TeamID"].fillna(0).astype(int)

In [251]:
Melo[Melo["TeamID"]==0]


,Team,Record,ELO,Rank,Normalized,TeamID
20,Saint Mary's College,27-5,1695.63,19,saintmaryscollege,0
157,FAU,17-15,1422.67,114,fau,0
213,Saint Thomas,21-10,1361.06,150,saintthomas,0
268,Southeast Missouri,18-12,1311.79,189,southeastmissouri,0
270,Seattle University,12-18,1310.88,191,seattleuniversity,0
313,Queens,17-14,1269.91,220,queens,0
337,UTA,11-18,1248.88,234,uta,0
383,Presbyterian College,11-18,1210.13,263,presbyteriancollege,0


In [252]:
import pandas as pd
from fuzzywuzzy import process

# fau & uta의 Normalized 값을 변경
Melo.loc[Melo["Normalized"] == "fau", "Normalized"] = "florida atlantic"
Melo.loc[Melo["Normalized"] == "uta", "Normalized"] = "texas arlington"
Melo.loc[Melo["Normalized"] == "southeastmissouri", "Normalized"] = "se missouri st"
#  발견되지 않은 팀(TeamID가 0인 팀) 리스트
unmatched_teams = Melo[Melo["TeamID"] == 0]["Normalized"].tolist()

# 기준이 되는 기존 팀 리스트 (MTeamname의 Normalized 컬럼)
existing_teams = MTeamname["Normalized"].tolist()

# 유사도 기반 자동 매칭
matched_results = []
for team in unmatched_teams:
    best_match, score = process.extractOne(team, existing_teams)
    matched_results.append((team, best_match, score))

# 매칭 결과를 DataFrame으로 변환
matched_df = pd.DataFrame(matched_results, columns=["Unmatched", "Matched", "Score"])

#  기존 팀 데이터와 ID 가져오기
matched_df = matched_df.merge(MTeamname[["Normalized", "TeamID"]], left_on="Matched", right_on="Normalized", how="left")
print(matched_df)
#Melo에 업데이트 적용
Melo = Melo.merge(matched_df[["Unmatched", "TeamID"]], left_on="Normalized", right_on="Unmatched", how="left", suffixes=("", "_new"))
Melo["TeamID"] = Melo["TeamID_new"].fillna(Melo["TeamID"]).astype(int)
Melo.drop(columns=["Unmatched", "TeamID_new"], inplace=True)  # 불필요한 컬럼 제거



              Unmatched          Matched  Score       Normalized  TeamID
0     saintmaryscollege       saintmarys     90       saintmarys    1388
1     saintmaryscollege       saintmarys     90       saintmarys    1388
2     saintmaryscollege       saintmarys     90       saintmarys    1388
3      florida atlantic  floridaatlantic     97  floridaatlantic    1194
4      florida atlantic  floridaatlantic     97  floridaatlantic    1194
5           saintthomas  saintthomasminn     85  saintthomasminn    1472
6        se missouri st     semissourist     92     semissourist    1369
7        se missouri st     semissourist     92     semissourist    1369
8     seattleuniversity          seattle     90          seattle    1370
9                queens         queensnc     86         queensnc    1474
10               queens         queensnc     86         queensnc    1474
11      texas arlington   texasarlington     97   texasarlington    1426
12      texas arlington   texasarlington     97   t

In [253]:
Welo[Welo["TeamID"] == 0]

,Team,Record,ELO,Rank,Normalized,TeamID
212,UTA,15-13,1358.91,145,uta,0
290,Saint Thomas,14-14,1283.63,199,saintthomas,0
294,Saint Mary's College,14-16,1279.12,202,saintmaryscollege,0
421,FAU,11-21,1160.74,285,fau,0
454,Queens,8-19,1089.95,306,queens,0
507,Southeast Missouri,5-23,1006.63,340,southeastmissouri,0
522,Presbyterian College,5-24,947.89,350,presbyteriancollege,0
524,Seattle University,3-26,942.77,352,seattleuniversity,0


In [254]:
import pandas as pd
from fuzzywuzzy import process

# fau & uta의 Normalized 값을 변경
Welo.loc[Welo["Normalized"] == "fau", "Normalized"] = "florida atlantic"
Welo.loc[Welo["Normalized"] == "uta", "Normalized"] = "texas arlington"
Welo.loc[Welo["Normalized"] == "southeastmissouri", "Normalized"] = "se missouri st"
#  발견되지 않은 팀(TeamID가 0인 팀) 리스트
unmatched_teams = Welo[Welo["TeamID"] == 0]["Normalized"].tolist()

# 기준이 되는 기존 팀 리스트 (MTeamname의 Normalized 컬럼)
existing_teams = WTeamname["Normalized"].tolist()

# 유사도 기반 자동 매칭
matched_results = []
for team in unmatched_teams:
    best_match, score = process.extractOne(team, existing_teams)
    matched_results.append((team, best_match, score))

# 매칭 결과를 DataFrame으로 변환
matched_df = pd.DataFrame(matched_results, columns=["Unmatched", "Matched", "Score"])

#  기존 팀 데이터와 ID 가져오기
matched_df = matched_df.merge(WTeamname[["Normalized", "TeamID"]], left_on="Matched", right_on="Normalized", how="left")
print(matched_df)
#Melo에 업데이트 적용
Welo = Welo.merge(matched_df[["Unmatched", "TeamID"]], left_on="Normalized", right_on="Unmatched", how="left", suffixes=("", "_new"))
Welo["TeamID"] = Welo["TeamID_new"].fillna(Welo["TeamID"]).astype(int)
Welo.drop(columns=["Unmatched", "TeamID_new"], inplace=True)  # 불필요한 컬럼 제거



              Unmatched          Matched  Score       Normalized  TeamID
0       texas arlington   texasarlington     97   texasarlington    3426
1       texas arlington   texasarlington     97   texasarlington    3426
2           saintthomas  saintthomasminn     85  saintthomasminn    3472
3     saintmaryscollege       saintmarys     90       saintmarys    3388
4     saintmaryscollege       saintmarys     90       saintmarys    3388
5     saintmaryscollege       saintmarys     90       saintmarys    3388
6      florida atlantic  floridaatlantic     97  floridaatlantic    3194
7      florida atlantic  floridaatlantic     97  floridaatlantic    3194
8                queens         queensnc     86         queensnc    3474
9                queens         queensnc     86         queensnc    3474
10       se missouri st     semissourist     92     semissourist    3369
11       se missouri st     semissourist     92     semissourist    3369
12  presbyteriancollege     presbyterian     90    

In [255]:
elo_df = pd.concat([Melo, Welo], ignore_index=True)


In [256]:
# 데이터 로드
sub = pd.read_csv('../input/march-machine-learning-mania-2025/SampleSubmissionStage2.csv')

# 먼저 T1_TeamID, T2_TeamID, Season을 생성해야 함
sub["Season"] = sub["ID"].apply(lambda x: x[:4]).astype(int)
sub["T1_TeamID"] = sub["ID"].apply(lambda x: x[5:9]).astype(int)
sub["T2_TeamID"] = sub["ID"].apply(lambda x: x[10:14]).astype(int)


In [257]:
elo_df.tail(5)

,Team,Record,ELO,Rank,Normalized,TeamID
1100,Arkansas-Pine Bluff,1-25,887.88,360,arkansaspinebluff,3115
1101,Long Island,6-24,884.35,361,longisland,3254
1102,Long Island,6-24,884.35,361,longisland,3254
1103,South Carolina State,2-29,856.91,362,southcarolinastate,3354
1104,South Carolina State,2-29,856.91,362,southcarolinastate,3354


In [258]:
sub = sub.merge(
    elo_df[['TeamID', 'ELO']].rename(columns={"ELO": "T1_ELO"}), 
    left_on="T1_TeamID", right_on="TeamID", how="left"
).drop(columns=["TeamID"])

sub = sub.merge(
    elo_df[['TeamID', 'ELO']].rename(columns={"ELO": "T2_ELO"}), 
    left_on="T2_TeamID", right_on="TeamID", how="left"
).drop(columns=["TeamID"])

print(sub.head())  # 최종 확인


               ID  Pred  Season  T1_TeamID  T2_TeamID   T1_ELO   T2_ELO
0  2025_1101_1102   0.5    2025       1101       1102  1291.73  1046.19
1  2025_1101_1102   0.5    2025       1101       1102  1291.73  1046.19
2  2025_1101_1102   0.5    2025       1101       1102  1291.73  1046.19
3  2025_1101_1102   0.5    2025       1101       1102  1291.73  1046.19
4  2025_1101_1103   0.5    2025       1101       1103  1291.73  1569.75


In [259]:
print("T1_ELO NaN 행:")
print(sub[sub["T1_ELO"].isna()])

print("T2_ELO NaN 행:")
print(sub[sub["T2_ELO"].isna()])


T1_ELO NaN 행:
Empty DataFrame
Columns: [ID, Pred, Season, T1_TeamID, T2_TeamID, T1_ELO, T2_ELO]
Index: []
T2_ELO NaN 행:
Empty DataFrame
Columns: [ID, Pred, Season, T1_TeamID, T2_TeamID, T1_ELO, T2_ELO]
Index: []


In [260]:
missing_t1 = sub[~sub["T1_TeamID"].isin(elo_df["TeamID"])]
print(f"🚨 T1_TeamID가 elo_df에 없는 개수: {len(missing_t1)}")


🚨 T1_TeamID가 elo_df에 없는 개수: 0


In [263]:
# elo_df에 없는 T1_TeamID 목록
missing_t1_ids = list(set(sub["T1_TeamID"]) - set(elo_df["TeamID"]))

# elo_df에 없는 T2_TeamID 목록
missing_t2_ids = list(set(sub["T2_TeamID"]) - set(elo_df["TeamID"]))

print("🚨 elo_df에 없는 T1_TeamID 고유값 리스트:", missing_t1_ids)
print("🚨 elo_df에 없는 T2_TeamID 고유값 리스트:", missing_t2_ids)


🚨 elo_df에 없는 T1_TeamID 고유값 리스트: []
🚨 elo_df에 없는 T2_TeamID 고유값 리스트: []


In [271]:

# ELO 차이를 이용해 Pred 값을 업데이트하는 함수 정의
def update_pred_by_elo(df):
    """
    ELO 차이를 기반으로 Pred 값을 업데이트하는 함수
    - ELO 차이가 클수록 승률(Pred)이 높아짐
    - 로지스틱 함수 기반으로 승률을 계산하여 반영
    """
    import numpy as np
    
    # ELO 차이 계산 (T1이 T2보다 높은 경우 양수, 낮은 경우 음수)
    df["ELO_Diff"] = df["T1_ELO"] - df["T2_ELO"]
    
    # 로지스틱 변환을 이용하여 승률(Pred) 계산
    df["Pred"] = 1 / (1 + np.exp(-df["ELO_Diff"] / 400))  # 400은 일반적인 ELO 스케일링
    
    return df

# 데이터 업데이트 적용
updated_df = update_pred_by_elo(sub)

updated_df


,ID,Pred,Season,T1_TeamID,T2_TeamID,T1_ELO,T2_ELO,ELO_Diff
0,2025_1101_1102,0.648819,2025,1101,1102,1291.73,1046.19,245.54
1,2025_1101_1102,0.648819,2025,1101,1102,1291.73,1046.19,245.54
2,2025_1101_1102,0.648819,2025,1101,1102,1291.73,1046.19,245.54
3,2025_1101_1102,0.648819,2025,1101,1102,1291.73,1046.19,245.54
4,2025_1101_1103,0.332911,2025,1101,1103,1291.73,1569.75,-278.02
...,...,...,...,...,...,...,...,...
304290,2025_3477_3479,0.550409,2025,3477,3479,1039.73,958.80,80.93
304291,2025_3477_3480,0.418764,2025,3477,3480,1039.73,1170.87,-131.14
304292,2025_3478_3479,0.551690,2025,3478,3479,1041.80,958.80,83.00
304293,2025_3478_3480,0.420024,2025,3478,3480,1041.80,1170.87,-129.07


In [273]:
updated_df


,ID,Pred,Season,T1_TeamID,T2_TeamID,T1_ELO,T2_ELO,ELO_Diff


In [275]:
# ID와 Pred 컬럼만 추출한 데이터 생성
id_pred_df = updated_df[["ID", "Pred"]]

# CSV 파일로 저장
csv_file_path = "id_pred.csv"  # 원하는 경로로 변경 가능
id_pred_df.to_csv(csv_file_path, index=False)

print(f" CSV 파일이 저장되었습니다: {csv_file_path}")


 CSV 파일이 저장되었습니다: id_pred.csv


🔍 T1_TeamID 고유값 개수: 724
🔍 T2_TeamID 고유값 개수: 724
🔍 elo_df에 존재하는 TeamID 개수: 726
🔍 elo_df에 없는 전체 TeamID 개수: 0


🔍 elo_df에서 중복된 TeamID 개수: 379


                   Team Record      ELO  Rank        Normalized  TeamID
292   Abilene Christian  13-16  1291.73   203  abilenechristian    1101
291   Abilene Christian  13-16  1291.73   203  abilenechristian    1101
503           Air Force   4-28  1046.19   336          airforce    1102
502           Air Force   4-28  1046.19   336          airforce    1102
529         Alabama A&M   7-22   986.08   352         alabamaam    1105
...                 ...    ...      ...   ...               ...     ...
985    Youngstown State   9-19  1148.23   288   youngstownstate    3464
740        UC San Diego  18-15  1394.89   130        ucsandiego    3471
741        UC San Diego  18-15  1394.89   130        ucsandiego    3471
1013             Queens   8-19  1089.95   306            queens    3474
1012             Queens   8-19  1089.95   306            queens    3474

[726 rows x 6 columns]


Empty DataFrame
Columns: [Team, Record, ELO, Rank, Normalized, TeamID]
Index: []


             Team Record      ELO  Rank  Normalized  TeamID
740  UC San Diego  18-15  1394.89   130  ucsandiego    3471
741  UC San Diego  18-15  1394.89   130  ucsandiego    3471


TeamID
1167    3
1379    3
1401    3
3386    3
3275    3
       ..
1340    2
1329    2
1421    2
1245    2
1450    2
Name: count, Length: 347, dtype: int64


# Now Women Part
